### Preparation des données 

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from xgboost import XGBRegressor

In [2]:
df = pd.read_excel('dataframe.xlsx')
df.head()

,Year,Inflation,Investment,Avoirs_Exterieurs_en_millions_mro,Taux_de_change_mensuel_en_dollar,Australie,Canada,Chine,Espagne,France,Italie,Japon,Mauritanie,Exportations_en_millions_mro,M2_en_millions_mro
0,1992,10.141818,0.348815,7484,190.0,0.427678,0.247550,3.257417,0.141241,0.051179,0.048388,0.026117,1.874126,53369.858,26904
1,1993,9.370344,0.871081,5567,191.0,4.045157,0.738843,3.179374,-0.156787,-0.020117,-0.049463,-0.013317,5.873637,58919.858,27172
2,1994,4.128259,0.106933,3943,192.0,3.979706,1.249346,2.985429,0.362246,0.075469,0.124759,0.031418,-3.060732,59775.858,30850
3,1995,6.543791,0.334144,7248,193.0,3.883165,0.745687,2.508456,0.419139,0.067412,0.167437,0.076299,9.819800,64622.858,29302
4,1996,4.681306,-0.020428,8357,196.0,3.863934,0.469531,2.272265,0.404405,0.045216,0.073474,0.090882,5.818827,67257.858,34264


In [3]:
df = df.replace(',', '.', regex=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 15 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Year                               32 non-null     int64  
 1   Inflation                          32 non-null     float64
 2   Investment                         32 non-null     float64
 3   Avoirs_Exterieurs_en_millions_mro  32 non-null     int64  
 4   Taux_de_change_mensuel_en_dollar   32 non-null     float64
 5   Australie                          32 non-null     float64
 6   Canada                             32 non-null     float64
 7   Chine                              32 non-null     float64
 8   Espagne                            32 non-null     float64
 9   France                             32 non-null     float64
 10  Italie                             32 non-null     float64
 11  Japon                              32 non-null     float64
 

### Modelisation 

On utilise les donnes $X_t-1$ cars les donne sur l'inflantion la croissance economique n'est d'isponible que lors de la fin de l'exercice

notre modele sera donc du type :\
$Y_t = f(Y_t-1 , X_t-1)$
avec Y la varible cible 'Avoirs_Exterieurs'

### Regression MCO

In [6]:
base = df[['Inflation', 'Investment',
       'Taux_de_change_mensuel_en_dollar', 'Australie', 'Canada', 'Chine',
       'Espagne', 'France', 'Italie', 'Japon', 'Mauritanie',
       'Exportations_en_millions_mro', 'M2_en_millions_mro']]
X_t = base.copy()
X_l1 = X_t.shift(1).dropna()
Y_t = df['Avoirs_Exterieurs_en_millions_mro'][:-1]
Y_l1 = df['Avoirs_Exterieurs_en_millions_mro'].shift(1).dropna()

In [12]:
X = sm.add_constant(X_l1)
model = sm.OLS(Y_t.values.reshape(-1,1), X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.973
Model:                            OLS   Adj. R-squared:                  0.952
Method:                 Least Squares   F-statistic:                     47.05
Date:                Thu, 24 Oct 2024   Prob (F-statistic):           1.04e-10
Time:                        01:19:31   Log-Likelihood:                -369.08
No. Observations:                  31   AIC:                             766.2
Df Residuals:                      17   BIC:                             786.2
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const   

On ne peut pas utiliser une regression du modele MCO pour faire l'analyse 
Les coefficients des varibles ne sont pas significatives sauf celui de la croissance de l'Espagne , de la masse monetaire M2 et Taux_de_change_en_dollar

**Les propriétés des résidus**

- **Omnibus et Prob(Omnibus)** :
  - Le test Omnibus évalue la normalité des résidus en prenant en compte à la fois l'asymétrie (*skewness*) et l'aplatissement (*kurtosis*). La statistique de test Omnibus est de 0,904.
  - La *Prob(Omnibus)* de 0,636 indique une probabilité relativement élevée, ce qui signifie que l'on ne rejette pas l'hypothèse nulle de normalité des résidus. Les résidus semblent suivre une distribution normale.

- **Durbin-Watson** :
  - Le test de Durbin-Watson mesure l'autocorrélation des résidus. Les valeurs varient de 0 à 4, où 2 indique l'absence d'autocorrélation.
  - Avec une valeur de 1,576, il y a une légère autocorrélation positive des résidus, mais ce n'est pas suffisamment élevé pour être préoccupant. Cela suggère qu'il n'y a pas de problème majeur d'autocorrélation, bien qu'une vérification plus approfondie puisse être utile.

- **Jarque-Bera (JB) et Prob(JB)** :
  - Le test de Jarque-Bera évalue également la normalité en examinant l'asymétrie et l'aplatissement des résidus.
  - Avec une statistique de test de 0,834 et une *Prob(JB)* de 0,659, les résultats indiquent que l'hypothèse de normalité ne peut pas être rejetée, renforçant l'idée que les résidus sont normalement distribués.

- **Skew (Asymétrie)** :
  - L'asymétrie mesure la symétrie des résidus par rapport à leur moyenne. Une valeur proche de zéro suggère une symétrie.
  - Avec une *skew* de -0,157, il y a une légère asymétrie négative, mais la valeur est très proche de zéro, ce qui ne pose pas de problème significatif pour l'analyse.

- **Kurtosis (Aplatissement)** :
  - La kurtosis mesure le degré d'aplatissement de la distribution des résidus. Une valeur de 3 indique une distribution normale (mésokurtique).
  - Avec une kurtosis de 2,260, les résidus montrent une légère platicurticité (moins de données dans les queues de la distribution que dans une distribution normale). Cela n'indique pas de problème majeur de normalité.

- **Cond. No. (Condition Number)** :
  - Le numéro de condition reflète la colinéarité potentielle entre les variables indépendantes. Des valeurs très élevées (supérieures à $(10^5)$) suggèrent une colinéarité importante.
  - Ici, le *Cond. No.* est de $(5,20 \times 10^7)$, ce qui est une valeur elevé, indiquant une colinéarité entre les variables explicatives. Cela suggère que les estimations des coefficients du modèle ne sont pas fiables.
  
  
  
En résumé, les tests de diagnostic montrent que les résidus suivent une distribution normale, qu'il existe un problème significatif d'autocorrélation, et que la colinéarité entre les variables est présente. Cela indique que, bien que le modèle soit conforme à certaines hypothèses de la régression linéaire, des ajustements sont nécessaires pour améliorer la fiabilité des résultats.


### Regression penalisée 

In [9]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, RegressorMixin

On a utilsé la validation croisée pour selectioner les parametre sachant que la formule de 
$\hat{\beta} = \underset{\beta}{\arg\min} \left( \frac{1}{2n} \sum_{i=1}^{n} (y_i - \mathbf{x}_i^\top \beta)^2 + \lambda_1 \|\beta\|_1 + \lambda_2 \|\beta\|_2^2 \right)$ 


In [35]:
import numpy as np
import statsmodels.api as sm
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

class StatsmodelsOLS(BaseEstimator, RegressorMixin):
    def __init__(self, alpha=0.0, L1_wt=0.5):
        self.alpha = alpha
        self.L1_wt = L1_wt
        self.model = None
    
    def fit(self, X, y):
        self.model = sm.OLS(y, X).fit_regularized(method='elastic_net', alpha=self.alpha, L1_wt=self.L1_wt)
        return self
    
    def predict(self, X):
        return self.model.predict(X)
    
    def aic(self):
        """Calculer l'AIC du modèle."""
        if self.model is not None:
            return self.model.aic
        else:
            raise ValueError("Le modèle n'est pas encore ajusté.")

# Définir une fonction de scoring personnalisée pour AIC
def aic_scorer(estimator, X, y):
    estimator.fit(X, y)
    return estimator.aic

# Définir le modèle
model = StatsmodelsOLS()

# Définir les paramètres à explorer
param_grid = {
    'alpha': [0.0, 0.1, 1.0, 10.0],  # Ajoutez d'autres valeurs selon vos besoins
    'L1_wt': [0.0, 0.5, 1.0]         # Ajustez selon vos besoins
}

# Créer l'objet GridSearchCV avec le score AIC
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring=make_scorer(aic_scorer, greater_is_better=False))

# Ajuster le modèle sur le jeu de données
grid_search.fit(Y_t.values.reshape(-1, 1), X_l1)

# Afficher les meilleurs paramètres
print("Meilleurs paramètres :", grid_search.best_params_)
print("Meilleure performance (AIC) :", grid_search.best_score_)


Meilleurs paramètres : {'L1_wt': 0.0, 'alpha': 0.0}
Meilleure performance (AIC) : nan


Les résultats indiquent que les meilleurs paramètres trouvés pour le modèle sont :

- `L1_wt = 0.0`
- `alpha = 0.0`

Cela signifie que le modèle optimal n'applique aucune régularisation, ni lasso (L1) ni ridge (L2). En d'autres termes, le modèle préfère ne pas imposer de pénalisation sur les coefficients.
On revien a la regression MCO .

### Modelisation Avec ACP et MCO

In [28]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


# Étape 1 : Normalisation des données
scaler = StandardScaler()
data_normalized = scaler.fit_transform(X_l1)

# Étape 2 : Réalisation de l'ACP
pca = PCA(n_components=3)  # Choisir le nombre de composantes principales
principal_components = pca.fit_transform(data_normalized)

# Étape 3 : Extraction des résultats
# Coordonnées des points sur les nouveaux axes
coordonnees_points = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2' , 'PC3'])

# Composantes des axes
composantes_axes = pd.DataFrame(pca.components_, columns=X_l1.columns, index=['PC1', 'PC2','PC3'])

# Variance expliquée par chaque composante
variance_expliquee = pca.explained_variance_ratio_

# Affichage des résultats
print("Coordonnées des points sur les nouveaux axes :\n", coordonnees_points)
print("\nComposantes des axes :\n", composantes_axes)
print("\nVariance expliquée par chaque composante :\n", variance_expliquee)


Coordonnées des points sur les nouveaux axes :
          PC1       PC2       PC3
0  -0.714999 -2.500877  0.677985
1  -0.930018 -2.887322  1.065160
2  -2.137604 -1.485773 -1.501151
3  -2.222559 -1.025666  0.560406
4  -1.505788 -1.205103 -0.185022
5  -1.811327 -1.062822 -1.777770
6  -2.031467 -0.972706 -0.609651
7  -2.110918 -0.475189 -1.213908
8  -2.495924  0.282293 -1.990161
9  -0.405239 -0.633898 -0.993252
10 -0.550844 -0.812631 -0.826126
11 -0.542897 -0.834588  0.400741
12 -1.862372 -0.123046  1.980514
13 -1.495506 -0.108497  3.726725
14 -1.366700 -0.014146  1.840388
15 -1.432765 -0.753113 -0.262829
16  0.621417 -1.219130  0.100676
17  4.293551 -3.205065 -1.108242
18 -0.538499  0.389789  0.175272
19  0.569200  0.412242  0.434882
20  1.612296  0.328270  1.621532
21  1.523696  0.790386  0.971134
22  1.028111  0.839799 -0.158494
23  1.020999  1.145791 -0.035465
24  1.244098  1.225535 -1.259393
25  0.749289  2.218829 -0.199049
26  1.113736  2.037902 -0.091121
27  1.924444  1.735082 -2.12

In [30]:
X = coordonnees_points[['PC1' , 'PC2']]
X = sm.add_constant(X)
model = sm.OLS(Y_t.values.reshape(-1,1), X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.856
Model:                            OLS   Adj. R-squared:                  0.846
Method:                 Least Squares   F-statistic:                     83.25
Date:                Sat, 26 Oct 2024   Prob (F-statistic):           1.64e-12
Time:                        00:20:44   Log-Likelihood:                -395.00
No. Observations:                  31   AIC:                             796.0
Df Residuals:                      28   BIC:                             800.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.646e+05   1.56e+04     10.529      0.0

**Les propriétés des résidus**

**Omnibus et Prob(Omnibus)** :

- L'Omnibus est un test combiné de normalité des résidus, basé à la fois sur la *skewness* (asymétrie) et la *kurtosis* (aplatissement). La statistique de test Omnibus est de 11,571 .
- La *Prob(Omnibus)* de 0,003 représente la probabilité associée au test Omnibus. Une valeur inferieure à 0,05 indique que l'on rejete l'hypothèse nulle selon laquelle les résidus suivent une distribution normale. Dans ce cas, les résidus ne semblent pas être normalement distribués.

**Durbin-Watson** :

- Le test Durbin-Watson vérifie la présence d'autocorrélation dans les résidus. Sa valeur varie entre 0 et 4, où 2 indique l'absence d'autocorrélation, une valeur proche de 0 indique une autocorrélation positive, et une valeur proche de 4 indique une autocorrélation négative.
- Ici, la statistique Durbin-Watson est de 0.637 , ce qui suggère une autocorrélation positive.

**Jarque-Bera (JB) et Prob(JB)** :

- Le test de Jarque-Bera évalue la normalité des résidus en utilisant les mesures de *skewness* et de *kurtosis*.
- La statistique de test est de 10.685 , et la *Prob(JB)* de 0.00478 indique que l'hypothèse de normalité doit être rejetée. Les résidus donc ne sont pas normalement distribués, ce qui est defavorable à l'hypothèse d'erreurs normales dans les régressions linéaires.

**Skew (Asymétrie)** :

- L'asymétrie mesure la symétrie des données. Une valeur proche de zéro indique une distribution symétrique.
- Avec une *skew* de 1.213 , les résidus montrent une asymétrie positive .

**Kurtosis (Aplatissement)** :

- La *kurtosis* mesure la concentration des données aux extrêmes. Une valeur de 3 indique une distribution normale (mésokurtique).
- Avec une *kurtosis* de 4.546, suggère une concentration élevée des résidus dans les queues.

**Cond. No. (Condition Number)** :

- Le numéro de condition indique la présence de colinéarité dans les variables indépendantes. Une valeur élevée (supérieure à $(10^5)$) peut indiquer une colinéarité élevée.
- Ici, un numéro de condition de 2.27 suggère qu'il n'y a pas de problème majeur de colinéarité dans le modèle.


Les résultats indiquent que les résidus du modèle ne suivent pas une distribution normale, présentent une autocorrélation positive, et montrent une asymétrie et une kurtosis qui suggèrent des valeurs extrêmes fréquentes. Ces violations des hypothèses classiques de la régression pourraient nécessiter des ajustements ou des méthodes alternatives pour garantir la validité des résultats.


### Modelisation avec ARIMAX

In [45]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from itertools import product


X = X_l1
y = Y_t.values.reshape(-1,1)


# Définition des plages de paramètres à tester
p_values = range(0, 3)
d_values = range(0, 2)
q_values = range(0, 3)

# Variables pour stocker les meilleurs résultats
best_aic = float("inf")
best_order = None
best_model = None

# Boucle pour tester les différentes combinaisons de paramètres
for p, d, q in product(p_values, d_values, q_values):
    try:
        # Ajustement du modèle ARIMAX
        model = sm.tsa.SARIMAX(y, order=(p, d, q), exog=X)
        result = model.fit(disp=False)
        
        # Vérification du critère AIC
        if result.aic < best_aic:
            best_aic = result.aic
            best_order = (p, d, q)
            best_model = result
    except:
        continue

# Affichage des meilleurs paramètres et du résumé du modèle
print(f"Meilleur modèle ARIMAX : ordre (p, d, q) = {best_order} avec AIC = {best_aic}")
print(best_model.summary())



Meilleur modèle ARIMAX : ordre (p, d, q) = (0, 1, 1) avec AIC = 748.5260361954125
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                   31
Model:               SARIMAX(0, 1, 1)   Log Likelihood                -359.263
Date:                Sat, 26 Oct 2024   AIC                            748.526
Time:                        02:08:36   BIC                            769.544
Sample:                             0   HQIC                           755.250
                                 - 31                                         
Covariance Type:                  opg                                         
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Inflation                        -1.101e+04   4845.728     -2.273      0.023   -2.05

***Interprétation des Résultats***

1. **Ljung-Box Test (Q)**:
   - **Statistique Q (L1)** : 0.02
   - **Probabilité (Prob(Q))** : 0.89
   - **Interprétation** : Le test de Ljung-Box teste l'hypothèse nulle selon laquelle il n'y a pas d'autocorrélation dans les résidus d'un modèle. Une valeur de probabilité (p-value) élevée (généralement supérieure à 0.05) indique que vous ne pouvez pas rejeter l'hypothèse nulle. Dans ce cas, avec un p-value de 0.89, cela signifie qu'il n'y a pas de preuves significatives d'autocorrélation dans les résidus.

2. **Jarque-Bera Test (JB)**:
   - **Statistique JB** : 1.57
   - **Probabilité (Prob(JB))** : 0.46
   - **Interprétation** : Le test de Jarque-Bera évalue la normalité des résidus. Une p-value élevée (comme 0.46 ici) signifie que vous ne pouvez pas rejeter l'hypothèse nulle de normalité, suggérant que les résidus suivent une distribution normale.

3. **Heteroskedasticity Test (H)**:
   - **Statistique H** : 2.80
   - **Probabilité (Prob(H) (two-sided))** : 0.12
   - **Interprétation** : Ce test examine la présence d'hétéroscédasticité dans les résidus. Une p-value de 0.12 indique qu'il n'y a pas de preuves suffisantes pour conclure à l'hétéroscédasticité, ce qui signifie que la variance des erreurs semble constante.

4. **Skewness**:
   - **Skew** : 0.53
   - **Interprétation** : Le skewness mesure l'asymétrie de la distribution des résidus. Un skew positif (0.53) indique que la distribution est légèrement asymétrique vers la droite.

5. **Kurtosis**:
   - **Kurtosis** : 2.65
   - **Interprétation** : La kurtosis mesure la "pointueté" de la distribution. Une kurtosis de 3 est considérée comme normale, donc ici, avec 2.65, cela suggère une distribution légèrement moins pointue que celle d'une distribution normale (c'est-à-dire plus aplatie).

### Conclusion
Dans l'ensemble, les résultats suggèrent que les résidus du modèle sont bien comportés, avec aucune preuve d'autocorrélation, une normalité acceptable et une homoscédasticité, ce qui est souhaitable pour de bonnes performances de modèle.
